<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=309ddae0f2581d25b80d9efae8cf8e1f29795532744abcec095aa6b92bd9a181
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-27 14:15:07
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.43 C
-------------------
Today PnL: -40.60 K (-0.28%)
Current PnL: -22.59 L (-14.8%)
CY Booked + Current PnL: -8.70 L (-5.7%)
-------------------
Total profit:  1.63 L
Total loss:  -24.22 L
-------------------
Total Booked + Current PnL: 18.28 L (14.55%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.7%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.00 L (61.45%)
Deployed:  1.26 C
Current:  1.43 C
CAGR/XIRR %: 7.43%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TTKPRESTIG,770.00,0.09,-14.04,16.44,0.09,14242.0,-14147.0,86630.0,101.07,50.0,M-SC,3.55,253.0,-0.99,0.60,13.81,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,-2.58,-9.57,10.60,0.02,16732.0,-16707.0,157852.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.30,OX40N,NTT,PAINTS
83,VOLTAS,1530.00,0.00,8.51,10.31,19.69,21450.0,16308.0,208050.0,-2.04,52.0,X-MC,3.36,78.0,0.76,1.45,15.88,XY25,NTT,AC
43,ITC,452.00,0.37,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.37,3.76,X40,NTT,FMCG
35,ICICIGI,2252.93,-1.57,5.56,13.86,20.19,25505.0,9699.0,184019.0,-17.20,46.0,X-MC,7.32,68.0,0.38,1.29,21.52,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
26,GILLETTE,11206.78,3.58,3.15,28.37,32.42,74301.0,8004.0,261900.0,-15.11,50.0,X-SC,7.55,85.0,0.11,1.83,18.73,X40,ATH,FMCG
22,DIXON,18931.72,2.94,-4.79,27.70,21.58,36959.0,-6714.0,133425.0,-52.01,38.0,X-MC,6.70,56.0,-0.18,0.93,18.12,X40N,ATH,IT
58,RAJOOENG,143.10,2.83,-25.31,86.91,39.61,66538.0,-25940.0,76560.0,-49.50,38.0,H-SC,20.50,135.0,-0.39,0.53,7.91,AR,ATH,MISC
67,SIS,528.00,2.46,-20.97,53.44,21.26,46891.0,-23286.0,87746.0,2074.09,61.0,H-SC,3.79,166.0,-0.50,0.61,18.55,OX40N,NTT,MISC
7,ATULAUTO,844.00,2.33,-18.66,78.97,45.57,126624.0,-36788.0,160344.0,3472.97,51.0,M-SC,4.03,244.0,-0.29,1.12,13.68,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WHIRLPOOL,2270.0,-10.99,-14.34,112.37,81.92,110504.0,-16459.0,98339.0,-49.86,23.0,M-SC,3.33,236.0,-0.15,0.69,16.42,XR,NTT,DURABLES
44,JCHAC,2282.0,-3.75,-27.63,38.16,-0.01,35296.0,-35310.0,92495.0,18834.78,16.0,M-SC,1.42,233.0,-1.00,0.65,4.74,OX40N,NTT,AC
39,INDIGOPNTS,1408.0,-2.58,-9.57,10.60,0.02,16732.0,-16707.0,157852.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.30,OX40N,NTT,PAINTS
28,GREENPANEL,537.0,-2.34,-32.78,112.93,43.14,138987.0,-60004.0,123074.0,175.84,32.0,M-SC,5.86,240.0,-0.43,0.86,15.77,XY24,NTT,MISC
70,SURYODAY,216.0,-1.87,-19.79,53.40,23.03,76696.0,-35445.0,143626.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.51,XR,NTT,BANKS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,0.02,0.55,105.58,106.72,130940.0,684.0,124020.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.87,24.06,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.21,-4.40,117.48,107.91,165821.0,-6496.0,141148.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.99,0.93,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-2.58,-9.57,10.60,0.02,16732.0,-16707.0,157852.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.30,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.09,-14.04,16.44,0.09,14242.0,-14147.0,86630.0,101.07,50.0,M-SC,3.55,253.0,-0.99,0.60,13.81,OX40N,NTT,DURABLES
47,KANSAINER,340.00,1.28,-20.90,43.46,13.47,92700.0,-56367.0,213300.0,-67.53,39.0,H-SC,1.46,158.0,-0.61,1.49,8.46,XY24,NTT,PAINTS
67,SIS,528.00,2.46,-20.97,53.44,21.26,46891.0,-23286.0,87746.0,2074.09,61.0,H-SC,3.79,166.0,-0.50,0.61,18.55,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.78,1.82,69.18,72.27,119359.0,3086.0,172534.0,-12.43,57.0,M-LC,3.89,99.0,0.03,1.20,8.11,XR,NTT,IT
38,INDIAMART,4810.62,0.02,0.55,105.58,106.72,130940.0,684.0,124020.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.87,24.06,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.78,1.82,69.18,72.27,119359.0,3086.0,172534.0,-12.43,57.0,M-LC,3.89,99.0,0.03,1.20,8.11,XR,NTT,IT
38,INDIAMART,4810.62,0.02,0.55,105.58,106.72,130940.0,684.0,124020.0,-50.25,39.0,H-SC,3.09,139.0,0.01,0.87,24.06,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,-0.32,-1.14,37.19,35.63,77053.0,-2386.0,207188.0,-14.97,41.0,H-MC,3.97,119.0,-0.03,1.45,15.34,AR,ATH,PHARMA
25,FINCABLES,1641.55,-0.21,-4.40,117.48,107.91,165821.0,-6496.0,141148.0,-22.17,37.0,M-SC,10.56,216.0,-0.04,0.99,0.93,OX40N,ATH,CABLES
37,IEX,219.00,-0.64,-4.41,55.48,48.63,106041.0,-8821.0,191133.0,-36.48,52.0,H-SC,15.62,136.0,-0.08,1.33,6.63,XR,NTT,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.26,-35.31,108.99,35.20,90724.0,-45429.0,83241.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.26,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.04,-9.08,26.74,15.23,48707.0,-18198.0,182149.0,-22.32,28.0,X-MC,6.92,66.0,-0.37,1.27,20.24,X40N,NTT,REALTY
76,TMPV,600.00,-0.28,-15.11,67.48,42.18,158345.0,-41756.0,234654.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.65,XY24,NTT,AUTO
55,PGHH,17907.65,0.01,-4.56,40.16,33.76,76969.0,-9165.0,191655.0,-32.64,33.0,X-MC,3.73,57.0,-0.12,1.34,0.40,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.57,-13.63,71.88,48.45,124485.0,-27322.0,173184.0,-19.87,34.0,X-MC,7.31,64.0,-0.22,1.21,0.61,X40N,ATH,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.37,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.37,3.76,X40,NTT,FMCG
76,TMPV,600.0,-0.28,-15.11,67.48,42.18,158345.0,-41756.0,234654.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.65,XY24,NTT,AUTO
3,ACC,3906.0,-0.84,-21.47,109.02,64.15,203726.0,-51081.0,186870.0,-54.13,53.0,X-SC,1.49,82.0,-0.25,1.30,4.82,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,0.58,3.70,26.30,30.97,48201.0,6534.0,183272.0,-20.69,59.0,X-MC,1.83,75.0,0.14,1.28,16.72,X40,ATH,INSURANCE
66,SIEMENS,4671.5,-0.87,-11.60,41.99,25.51,69074.0,-21595.0,164500.0,0.93,63.0,H-LC,1.88,49.0,-0.31,1.15,21.22,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,0.26,-35.31,108.99,35.20,90724.0,-45429.0,83241.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.26,X40,NTT,FOOTWEAR
55,PGHH,17907.65,0.01,-4.56,40.16,33.76,76969.0,-9165.0,191655.0,-32.64,33.0,X-MC,3.73,57.0,-0.12,1.34,0.40,X40,ATH,FMCG
9,BAJAJHFL,181.50,0.57,-13.63,71.88,48.45,124485.0,-27322.0,173184.0,-19.87,34.0,X-MC,7.31,64.0,-0.22,1.21,0.61,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.51,-5.22,33.00,26.07,51216.0,-8540.0,155200.0,-30.18,38.0,X-MC,9.38,55.0,-0.17,1.08,0.69,X40,ATH,APPARELS
76,TMPV,600.00,-0.28,-15.11,67.48,42.18,158345.0,-41756.0,234654.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.65,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-0.19,-27.74,97.69,42.84,45886.0,-18035.0,46971.0,-52.62,41.0,X-SC,37.56,83.0,-0.39,0.33,2.21,XY24,NTT,MISC
60,RELAXO,1176.00,-0.74,-46.98,187.28,52.31,144127.0,-68202.0,76958.0,-43.57,37.0,X-SC,4.44,91.0,-0.47,0.54,3.01,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,0.26,-35.31,108.99,35.20,90724.0,-45429.0,83241.0,9.17,26.0,X-SC,13.18,92.0,-0.50,0.58,0.26,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-1.52,3.88,19.44,24.08,25636.0,4925.0,131875.0,-4.53,47.0,X-SC,5.57,89.0,0.19,0.92,24.26,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,2.94,-4.79,27.70,21.58,36959.0,-6714.0,133425.0,-52.01,38.0,X-MC,6.70,56.0,-0.18,0.93,18.12,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,-1.10,-15.01,40.34,19.28,118613.0,-51924.0,294032.0,-25.89,57.0,X-LC,7.24,1.0,-0.44,2.05,8.70,X40,ATH,IT
41,INFY,2275.00,0.03,6.09,45.98,54.87,152626.0,19051.0,331939.0,-17.44,61.0,X-LC,2.63,2.0,0.12,2.32,14.88,X40,BTT,IT
76,TMPV,600.00,-0.28,-15.11,67.48,42.18,158345.0,-41756.0,234654.0,-24.30,28.0,X-LC,1.48,3.0,-0.26,1.64,1.65,XY24,NTT,AUTO
82,VBL,671.64,0.44,-5.55,43.65,35.67,130207.0,-17545.0,298297.0,-16.71,55.0,X-LC,5.76,4.0,-0.13,2.08,7.53,X40N,ATH,FMCG
43,ITC,452.00,0.37,-1.71,12.35,10.44,24296.0,-3413.0,196725.0,-41.15,42.0,X-LC,0.89,5.0,-0.14,1.37,3.76,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.53,-36.86,110.48,32.89,53855.0,-28460.0,48746.0,-701.92,65.0,L-MC,5.64,259.0,-0.53,0.34,34.24,XR,NTT,BANKS
6,ASIANTILES,137.00,-0.74,-13.30,109.54,81.67,89092.0,-12477.0,81333.0,7311.11,50.0,L-SC,19.27,271.0,-0.14,0.57,57.50,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.84,-11.52,57.71,39.54,51653.0,-11655.0,89504.0,-30.76,68.0,M-SC,6.61,220.0,-0.23,0.63,26.21,AR,ATH,AUTO
51,MASFIN,398.61,-0.89,-6.69,30.80,22.05,28159.0,-6555.0,91425.0,-19.60,45.0,H-SC,6.94,164.0,-0.23,0.64,32.89,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.87,-19.79,53.40,23.03,76696.0,-35445.0,143626.0,55.70,48.0,H-SC,9.45,167.0,-0.46,1.00,42.51,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-2.58,-9.57,10.60,0.02,16732.0,-16707.0,157852.0,140.51,65.0,M-SC,9.11,234.0,-1.00,1.10,37.30,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,0.19,0.65,20.17,20.95,51110.0,1628.0,253396.0,-0.35,76.0,X-LC,14.55,31.0,0.03,1.77,36.51,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,0.53,-36.86,110.48,32.89,53855.0,-28460.0,48746.0,-701.92,65.0,L-MC,5.64,259.0,-0.53,0.34,34.24,XR,NTT,BANKS
0,5PAISA,593.00,-0.79,-35.05,73.82,12.90,104764.0,-76578.0,141918.0,109.13,58.0,H-SC,13.41,161.0,-0.73,0.99,18.00,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,-0.92,6.22,13.69,20.77,34049.0,14568.0,248712.0,-3.08,61.0,X-MC,3.76,62.0,0.43,1.74,13.57,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.81
1,25,44.73
2,50,75.55


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.56
MC    29.23
LC    25.21
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.85
X40      21.22
X40N     11.94
XR        9.51
AR        9.12
XY25      9.07
OX40N     7.66
X200      1.88
SR        0.98
MH        0.77
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.92
X-MC    22.75
X-LC    20.01
M-SC    11.95
X-SC     8.28
H-MC     4.77
H-LC     3.03
L-SC     1.41
M-MC     1.37
M-LC     1.20
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.80,-4.58,38.40
IT,13.79,-14.56,73.70
FINANCE,9.44,-14.71,63.18
MISC,7.42,-22.11,72.14
ELECTRICAL,5.99,-9.50,49.61
PAINTS,5.97,-8.01,24.05
INSURANCE,4.43,0.27,34.88
PHARMA,3.88,-0.54,32.72
AUTO,3.39,-18.62,69.48


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3049368.0,21
XR,1299248.0,13
AR,1268377.0,10
X40,999853.0,14
X40N,737265.0,9
OX40N,695456.0,10
XY25,340881.0,6
SR,280229.0,2
MH,73976.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3551490.0,25
M-SC,1348593.0,15
X-MC,1225592.0,16
X-LC,896723.0,11
X-SC,733210.0,8
H-MC,397349.0,3
L-SC,260289.0,3
H-LC,124541.0,2
M-LC,119359.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1214903.0      6
           AR         871049.0      5
           XR         793922.0      7
M-SC       XY24       778303.0      6
X-MC       X40        440858.0      7
X-LC       X40        393970.0      5
X-MC       XY24       363270.0      3
H-SC       OX40N      317411.0      4
X-SC       X40N       292937.0      3
M-SC       OX40N      290234.0      5
H-SC       SR         280229.0      2
X-SC       XY24       275248.0      3
X-MC       X40N       244200.0      4
X-LC       XY24       228127.0      2
H-MC       AR         207832.0      2
X-LC       X40N       200128.0      2
H-MC       XY24       189517.0      1
X-MC       XY25       177264.0      2
L-SC       XR         172478.0      2
X-SC       X40        165025.0      2
M-SC       XR         159634.0      2
           AR         120422.0      2
M-LC       XR         119359.0      1
L-SC       OX40N       87811.0      1
X-LC       XY25        74498.0      2
H-SC       MH          73976.0      1
H-LC       AR          69074.0      1
           X200        55467.0      1
L-MC       XR          53855.0      1
M-MC       XY25        49279.0      1
L-LC       XY25        39840.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 62.0 seconds
